# CKY Parsing

## By Brea Koenes

### Overview

Building a simple parser that is trained from the ATIS (Air Traffic Information System) portion of the Penn Treebank. The training data consists of short queries and commands spoken by users of a fake robot travel agent.
The files provided and their purpose are listed below:

    - `train.trees.pre.unk` contains the trees of the training data: binary branching trees with all words that only appear once replaced with `<unk>`  
    - `dev.strings` contains the Strings of the development data

## CKY Parsing

CKY Parsing, named after its inventors, John Cocke, Daniel Younger, and Tadao Kasami, is a widely used algorithm for parsing sentences in context-free grammars. It stands out for its efficiency and capability to handle ambiguous grammatical structures, making it a fundamental tool in NLP.

## Importance in NLP

Syntactic parsing, the process of analyzing a sentence's structure according to the rules of a formal grammar, plays a critical role in understanding language. The CKY algorithm, with its bottom-up parsing approach, allows us to decompose complex sentences into their constituent parts, facilitating tasks like machine translation, sentiment analysis, and information extraction. 

## 01 - Setup

Set up our environment for the CKY Parsing exercise. This includes importing necessary libraries and preparing the environment for parsing.

In [25]:
import nltk
from nltk.tree import Tree
from nltk.parse import ViterbiParser
import svgling
import math

## 02 - Importing and Visualizing the Syntax Tree

### Syntax Tree

A syntax tree, also known as a parse tree, is a tree representation of the syntactic structure of a sentence. In linguistics and NLP, these trees are crucial for understanding how different parts of a sentence relate to each other according to the rules of grammar.

### Importance of Syntax Trees in NLP

Syntax trees play a vital role in various NLP applications. They help in:

- **Understanding Sentence Structure:** By breaking down a sentence into its constituent parts, syntax trees make it easier to analyze the grammatical structure.
- **Improving Language Models:** They are essential in developing more accurate models for tasks like machine translation, speech recognition, and sentiment analysis.
- **Information Extraction:** Syntax trees aid in extracting meaningful information from complex sentences, which is crucial in tasks like question answering and text summarization.

### Task: Importing and Visualizing the Syntax Tree

Import a syntax tree from the provided file `train.trees.pre.unk` and visualize its structure. 

1. **Read the First Line of the File:** Use Python's built-in `open` function to access the file. Read the first line of the file, which contains a sentence's syntax tree in a specific format. 

2. **Parse the Syntax Tree:** With the line read from the file, parse it to create a syntax tree. Store this object.

3. **Print the partial Syntax Tree:** Print it using the `pretty_print` method from the `first_tree` object.

In [35]:
# Open file and read first line
with open('train.trees.pre.unk', 'r') as file:
    first_line = file.readline().strip()

# Parse syntax tree
first_tree = Tree.fromstring(first_line)

# Print tree
first_tree.pretty_print()

                                        TOP                                                         
                 ________________________|_______________________________________________________    
               S_VP                                                                              |  
  ______________|________________________                                                        |   
 |                                       NP                                                      |  
 |         ______________________________|_________                                              |   
 |        |                                       NP*                                            |  
 |        |                 _______________________|_______                                      |   
 |        |                |                              NP*                                    |  
 |        |                |                  _____________|___________                

## 03 - Convert Trees to Chomsky Normal Form (CNF) and Extract Productions

Transform our syntax trees into Chomsky Normal Form (CNF) and then extract grammatical productions from these CNF trees. This process is crucial for creating a grammar that can be used for parsing.

### Chomsky Normal Form

Chomsky Normal Form is a way of rewriting and simplifying the grammar of a language. A grammar in CNF has its production rules restricted to a certain standard form. Specifically, each rule must be either:
- A -> BC, where 'A', 'B', and 'C' are non-terminal symbols, and 'B' and 'C' are not the start symbol.
- A -> a, where 'A' is a non-terminal symbol and 'a' is a terminal symbol.
This form is particularly useful for parsing algorithms and simplifying the analysis of grammatical structures.

### Task: Convert Trees to Chomsky Normal Form (CNF) and Extract Productions

1. **Read Syntax Trees from File:**
   Read all the syntax trees from the provided file `train.trees.pre.unk` into a list.

2. **Convert Trees to CNF:**
    Convert each tree into Chomsky Normal Form.

3. **Extract Productions:**

   Having converted the trees to Chomsky Normal Form, our next step is to extract the grammatical productions from these CNF trees. Grammatical productions are essentially the rules that define how sentences in a language can be constructed and are pivotal for understanding the structure and syntax of the language.

   To accomplish this, create a list named `productions`. This list will store all the production rules extracted from each tree. Traverse through each of the CNF trees, and for every tree, use the `productions()` method. This method will break down the tree into its constituent production rules. Accumulate these rules in the `productions` list, which will later be used for constructing our grammar.


In [36]:
import nltk
from nltk import Tree

trees = []
with open('train.trees.pre.unk', 'r') as file:
    for line in file:
        # Read each line, strip whitespace, parse into tree object
        tree = Tree.fromstring(line.strip())
        trees.append(tree)

# Convert each tree to Chomsky Normal Form
cnf_trees = []
for tree in trees:
    tree.chomsky_normal_form(horzMarkov=2)
    cnf_trees.append(tree)

# Extract productions from all CNF trees
productions = []
for tree in cnf_trees:
    productions.extend(tree.productions())

## 04 - Inducing a Probabilistic Context-Free Grammar (PCFG) from Productions

Induce a Probabilistic Context-Free Grammar (PCFG) from these productions. This step is crucial in the realm of syntactic parsing in NLP.

### What is a Probabilistic Context-Free Grammar (PCFG)?

A Probabilistic Context-Free Grammar (PCFG) is an extension of the standard context-free grammar that associates probabilities with its production rules. In PCFG, each production rule is assigned a probability that signifies how frequently that rule is used to generate a sentence in a language. This probabilistic aspect allows us to model linguistic phenomena more realistically and handle ambiguities in natural language more effectively.

### Importance of PCFG in NLP

In NLP, PCFGs are instrumental for several reasons:
- **Handling Ambiguity:** They help in dealing with ambiguities inherent in natural languages by preferring more likely structures over less likely ones.
- **Improving Parsing Accuracy:** PCFGs enhance the accuracy of parsing algorithms, making them more effective in understanding complex sentences.
- **Application in Various NLP Tasks:** They are fundamental in tasks such as machine translation, speech recognition, and syntactic analysis.

### Steps to Induce a PCFG from Productions:

1. **Define the Start Symbol:**
   First, identify the start symbol for grammar. Create a nonterminal symbol using `nltk.Nonterminal` passing the start symbol as paremeter.

2. **Induce the PCFG:**
    Utilize the `induce_pcfg` function from NLTK to create a PCFG grammer based on the start symbol and the list of **productions**.

3. **Display Sample Grammar Rules:**
To get a sense of what our induced grammar looks like, print out a sample of the grammar rules. Display the **first 10 rules** as an example.

In [28]:
# Define start symbol for grammar
start_symbol = nltk.Nonterminal('TOP')

# Introduce PCFG using start symbol and list of productions
grammar = nltk.induce_pcfg(start_symbol, productions)

# Display first 10 grammar rules
for i, production in enumerate(grammar.productions()[:10],1):
    print(production)

TOP -> S_VP PUNC [0.208955]
S_VP -> VB NP [0.122302]
VB -> 'List' [0.0180723]
NP -> NP NP* [0.115942]
NP -> DT NNS [0.0855072]
DT -> 'the' [0.626263]
NNS -> 'flights' [0.746411]
NP* -> PP NP* [0.121076]
PP -> IN NP_NNP [0.349927]
IN -> 'from' [0.452282]


## 05 - Import Sentences to parse

Create a list named `sentences` with all the sentences from 'dev.strings' file. Each line is a sentence.

In [29]:
# Create a list of sentences from dev_strings file
sentences = []
with open('dev.strings', 'r') as file:
    for line in file:
        sentences.append(line.strip())

## 06 - CKY Parsing with Unknown Word Handling

### Objective

Create a function named `cky_parsing` that applies the CKY algorithm to parse sentences using a given Probabilistic Context-Free Grammar (PCFG). The function should handle unknown words by substituting them with `<unk>` and should employ the Viterbi parsing algorithm. Additionally, it should account for the grammar's productions to identify known words.

### Instructions

1. **Function Definition:**
   - Name the function `cky_parsing`.
   - It should accept two parameters: `sentences` (a list of sentences to be parsed) and `grammar` (the PCFG used for parsing).

2. **Preprocessing:**
   - Within the function, construct a set of known words present in the given productions (generate 'production' set from the grammar passed). This set is used to determine if words in the sentences are covered by the grammar.
    
3. **Viterbi Parser Setup:**
   - Initialize a Viterbi parser using the provided PCFG.

4. **Sentence Processing:**
   - Iterate over each sentence in `sentences`:
     - Tokenize the sentence.
     - Replace any word not found in the set of known words with `<unk>`.
     - Parse the sentence using the Viterbi parser.
     - Select the parse with the highest probability, or handle cases where no valid parse is found (check the `parse_all` method of the ViterbiParser object from the `nltk.parse` library)


### Return Value

The function should return a list of tuples. One tuple per sentence processed. Each tuple should contain:
   - The index of the sentence within the input list.
   - The original sentence.
   - The best parse tree found, or an appropriate value indicating the grammatical structure of sentences. If there is no best parse found, set the best parse to `None`.


In [32]:
# Define function
def cky_parsing(sentences, grammar):
    results = []

    # Construct set of known words
    known_words = set()
    for production in grammar.productions():
        for rhs in production.rhs():
            if isinstance(rhs, str):
                known_words.add(rhs)

    # Initialize Viterbi parser with PCFG
    parser = nltk.ViterbiParser(grammar)

    # Iterate over each sentence with its index
    for idx, sentence in enumerate(sentences):
        tokens = sentence.split()

        # Replace unknown words with unk
        modified_tokens = [token if token in known_words else '<unk>' for token in tokens]
        
        # Parse modified sentence
        try:
            parse_trees = list(parser.parse_all(modified_tokens))
            if parse_trees:
                best_parse = parse_trees[0]
            else:
                best_parse = None
        except ValueError:
            best_parse = None

        # Add result tuple to list
        results.append((idx, sentence, best_parse))

    return results

## 07 - Executing CKY Parsing Function on Sentences

Execute the following cell to parse and print the parsing results. Do not change the code:

In [33]:
# Example of using the function (with 'sentences' being the list of sentences)
# Assuming 'productions' are the productions from the induced grammar
cky_charts = cky_parsing(sentences, grammar)

for index, sentence, chart in cky_charts:
    print(f"Line {index}: {sentence}")

    if chart:
        print(f"Probability: {math.log(chart.prob())}")  # Using log probability
        print("Parse:")
        chart.pretty_print()  # Pretty print the parse tree
    else:
        print("No parse available for this sentence")
    print("-------------------------------------------------")


Line 0: The flight should be eleven a.m tomorrow .
No parse available for this sentence
-------------------------------------------------
Line 1: I would like it to have a stop in New York and I would like a flight that serves breakfast .
Probability: -85.26919694246591
Parse:
                                                           TOP                                                                        
               _____________________________________________|______________________________________________________________________    
              S                                                                                                                    |  
   ___________|_______________                                                                                                     |   
  |                           VP                                                                                                   |  
  |       ____________________|______________

### 08 - Export Grammar

In [34]:
# Importing dill library for model serialization
import dill

# Serialization with dill
with open('grammar.dill', 'wb') as file:
    dill.dump(grammar , file)